In [1]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
grad_rates_1011 = pd.read_excel("data/grad_rates/Grad_Rates_2010-11_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1112 = pd.read_excel("data/grad_rates/Grad_Rates_2011-12_cohort.xls", sheet_name=1, usecols="B:H", skiprows=5)
grad_rates_1213 = pd.read_excel("data/grad_rates/Grad_Rates_2012-13_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1314 = pd.read_excel("data/grad_rates/Grad_Rates_2013-14_cohort.xlsx", sheet_name=1, usecols="B:H", skiprows=6)
grad_rates_1011.head(3)

,School ID,School Name,Cohort,Subgroup,Students,Grads,Rate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28.0,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3.0,s,s
2,101,John Bartram High School,2010-2011,All Students,281.0,203,0.72242


In [3]:
grad_rates_1112 = grad_rates_1112.rename(columns={'Subgrouop': 'Subgroup', '9th Grade Cohort': 'Cohort'})
grad_rates_1112 = grad_rates_1112[grad_rates_1112.Subgroup == 'All Students']

In [4]:
grad_rates = pd.concat([grad_rates_1011, grad_rates_1112, grad_rates_1213, grad_rates_1314]).reset_index(drop=True)
print(grad_rates.shape)
grad_rates.head()

(368, 7)


,School ID,School Name,Cohort,Subgroup,Students,Grads,Rate
0,31,Reti-Wrap Transition Center,2010-2011,All Students,28,5,0.178571
1,32,Family Support Center,2010-2011,All Students,3,s,s
2,101,John Bartram High School,2010-2011,All Students,281,203,0.72242
3,102,West Philadelphia High School,2010-2011,All Students,182,123,0.675824
4,103,High School Of The Future,2010-2011,All Students,146,104,0.712329


In [5]:
# replace 's' with NaN
# 's' means supressed
grad_rates = grad_rates.replace('s', np.nan)

# if grad count is suprressed then suppress student count too
grad_rates['Students'] = grad_rates['Students'].where(~pd.isnull(grad_rates['Grads']), np.nan)

# all subgroups are the same (equals 'All Students')
grad_rates.drop('Subgroup', axis=1, inplace=True)

In [6]:
grad_rates.head()

,School ID,School Name,Cohort,Students,Grads,Rate
0,31,Reti-Wrap Transition Center,2010-2011,28.0,5.0,0.178571
1,32,Family Support Center,2010-2011,NaN,NaN,NaN
2,101,John Bartram High School,2010-2011,281.0,203.0,0.722420
3,102,West Philadelphia High School,2010-2011,182.0,123.0,0.675824
4,103,High School Of The Future,2010-2011,146.0,104.0,0.712329


In [7]:
# the school names sometimes vary year-to-year
# clean them so we always take the latest spelling
def cleanNames(grp):
    name = grp.sort_values(by='Cohort', ascending=False)['School Name'].reset_index(drop=True)[0]
    return pd.Series({'Name_Cleaned': name})

name_cleaned = grad_rates.groupby('School ID').apply(cleanNames)

grad_rates = grad_rates.join(name_cleaned, on='School ID')
grad_rates.drop('School Name', axis=1, inplace=True)
grad_rates = grad_rates.rename(columns={'Name_Cleaned': 'School Name'})

In [8]:
# convert rows to columns
# get one row per school
grad_rates.set_index(['School ID', 'School Name', 'Cohort'], inplace=True)
grad_rates = grad_rates.unstack().reset_index()
grad_rates.columns = [' '.join(col).strip() for col in grad_rates.columns.values]
grad_rates.head()

,School ID,School Name,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,Rate 2010-2011,Rate 2011-2012,Rate 2012-2013,Rate 2013-2014
0,31,RETI-WRAP TRANSITION CENTER,28.0,NaN,10.0,14.0,5.0,NaN,1.0,1.0,0.178571,NaN,0.100000,0.071429
1,32,STUDENT TRANSITION-DEPENDENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101,JOHN BARTRAM HIGH SCHOOL,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645
3,102,WEST PHILADELPHIA HIGH SCHOOL,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068
4,103,HIGH SCHOOL OF THE FUTURE,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871


In [9]:
# df.columns = [' '.join(col).strip() for col in df.columns.values]
# df.head()

In [10]:
# get 4-year combined graduation rate
grad_rates['Students Total'] = grad_rates[['Students 2010-2011', 'Students 2011-2012', \
                                          'Students 2012-2013','Students 2013-2014']].sum(axis=1)
            
grad_rates['Grads Total'] = grad_rates[['Grads 2010-2011', 'Grads 2011-2012', \
                                        'Grads 2012-2013','Grads 2013-2014']].sum(axis=1)
            
grad_rates['GradRate Total'] = grad_rates['Grads Total'] / grad_rates['Students Total']
            
grad_rates.head()

,School ID,School Name,Students 2010-2011,Students 2011-2012,Students 2012-2013,Students 2013-2014,Grads 2010-2011,Grads 2011-2012,Grads 2012-2013,Grads 2013-2014,Rate 2010-2011,Rate 2011-2012,Rate 2012-2013,Rate 2013-2014,Students Total,Grads Total,Rate Total
0,31,RETI-WRAP TRANSITION CENTER,28.0,NaN,10.0,14.0,5.0,NaN,1.0,1.0,0.178571,NaN,0.100000,0.071429,52.0,7.0,0.134615
1,32,STUDENT TRANSITION-DEPENDENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,101,JOHN BARTRAM HIGH SCHOOL,281.0,293.0,239.0,217.0,203.0,177.0,130.0,126.0,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476
3,102,WEST PHILADELPHIA HIGH SCHOOL,182.0,171.0,156.0,118.0,123.0,107.0,83.0,76.0,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415
4,103,HIGH SCHOOL OF THE FUTURE,146.0,159.0,101.0,124.0,104.0,127.0,84.0,91.0,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038


In [11]:
# subset columns
df = grad_rates[['School ID', 'School Name', 'Rate 2010-2011', 'Rate 2011-2012', 'Rate 2012-2013', \
                 'Rate 2013-2014', 'Students Total', 'Grads Total', 'Rate Total']]

df.set_index(['School ID'], inplace=True)

# teacher attendance
teach_attnd = pd.read_excel("data/Teacher_Attend_1516.xlsx", sheet_name=0)
teach_attnd.set_index(['School ID'], inplace=True)

df = df.join(teach_attnd)
df.drop(['School Year', 'ULCS', 'SDP Avg'], axis=1, inplace=True)
df.head()

,School Name,Rate 2010-2011,Rate 2011-2012,Rate 2012-2013,Rate 2013-2014,Students Total,Grads Total,Rate Total,Teacher Attd %
School ID,,,,,,,,,
31,RETI-WRAP TRANSITION CENTER,0.178571,NaN,0.100000,0.071429,52.0,7.0,0.134615,NaN
32,STUDENT TRANSITION-DEPENDENT,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
101,JOHN BARTRAM HIGH SCHOOL,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5
102,WEST PHILADELPHIA HIGH SCHOOL,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0
103,HIGH SCHOOL OF THE FUTURE,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4


# Get GPS Coordinates
For each school, use the address to find the Latitude and Longitude Coordinates. This will let us place the schools on a map.

In [12]:
# read SPR data
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_SY1617_School_Metric_Scores_20180206.xlsx', sheet_name=1)

spr = spr.rename(columns={'SRC School ID': 'School ID'})
spr.set_index(['School ID', 'Report'], inplace=True)

spr['Full Address'] = spr['Street Address'] + ', ' + spr['City'] + ', ' + spr['State'] + ' ' + spr['Zip Code'].astype(str)
spr.head(3)

,,School,Rpt Type Long,Street Address,City,State,Zip Code,Phone Number,Fax Number,Governance,Turnaround Model,...,FAFSA Tier,Student Survey College & Career Score,Student Survey College & Career Pts Earn,Student Survey College & Career Pts Poss,Student Survey College & Career Pct Earn,Student Survey College & Career Tier,Teach Effect Distinguished Score,Teach Effect Instruction Score,Teacher Attendance Score,Student Survey Teaching Score
School ID,Report,,,,,,,,,,,,,,,,,,,,,
101,HS,John Bartram High School,High School,2401 S 67th St,Philadelphia,PA,19142,215-400-8100,215-400-8101,District,NaN,...,WATCH,19,0.19,1,19,INTERVENE,15,21,56,47
102,HS,West Philadelphia High School,High School,4901 Chestnut St,Philadelphia,PA,19139,215-400-7900,215-400-7901,District,Turnaround,...,WATCH,27,0.27,1,27,WATCH,11,29,56,42
103,HS,High School of the Future,High School,4021 Parkside Ave,Philadelphia,PA,19104,215-400-7790,215-400-7791,District,NaN,...,REINFORCE,0,0,1,0,INTERVENE,16,13,50,0


In [13]:
# grab single row by slice
spr.loc[(403, 'HS'), :].head(6)

School            High School of Engineering and Science
Rpt Type Long                                High School
Street Address                          1600 W Norris St
City                                        Philadelphia
State                                                 PA
Zip Code                                           19121
Name: (403, HS), dtype: object

In [26]:
import time

In [25]:
geolocator = Nominatim()
cell_start = time.time()

lat = pd.Series({})
lon = pd.Series({})

for index, row in spr.iterrows():
    addr = row['Full Address']

    if index[0] not in lat:
        try:
            location = geolocator.geocode(addr)
        except GeocoderTimedOut:
            print("Geocoder Timed Out. Address =", addr)

        try:
            latitude = location.latitude
            longitude = location.longitude
        except AttributeError:
            latitude = np.nan
            longitude = np.nan

        lat = pd.concat([lat, pd.Series({index[0]: latitude})])
        lon = pd.concat([lon, pd.Series({index[0]: longitude})])
    
    else:
        continue
        
print('Cell Runtime:', round(time.time() - cell_start), 'seconds')

Geocoder Timed Out. Address = 1750 N 12th St, Philadelphia, PA 19122


In [17]:
print(lat.shape, lon.shape)
lat = lat.rename("Latitude")
lon = lon.rename("Longitude")

(294,) (294,)


In [19]:
df = df.join(lat)
df = df.join(lon)
df.head()

,School Name,Rate 2010-2011,Rate 2011-2012,Rate 2012-2013,Rate 2013-2014,Students Total,Grads Total,Rate Total,Teacher Attd %,Latitude,Longitude
School ID,,,,,,,,,,,
31,RETI-WRAP TRANSITION CENTER,0.178571,NaN,0.100000,0.071429,52.0,7.0,0.134615,NaN,NaN,NaN
32,STUDENT TRANSITION-DEPENDENT,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
101,JOHN BARTRAM HIGH SCHOOL,0.722420,0.604096,0.543933,0.580645,1030.0,636.0,0.617476,93.5,39.921470,-75.234018
102,WEST PHILADELPHIA HIGH SCHOOL,0.675824,0.625731,0.532051,0.644068,627.0,389.0,0.620415,94.0,39.958587,-75.219431
103,HIGH SCHOOL OF THE FUTURE,0.712329,0.798742,0.831683,0.733871,530.0,406.0,0.766038,91.4,39.974678,-75.204826


In [20]:
df.to_csv("test_data.csv", sep='|', index=True)